In [1]:
import numpy as np

In [2]:
import pandas as pd

In [3]:
import requests

In [4]:
import xlsxwriter

In [5]:
import math

In [6]:
import pandas as pd 
stocks = pd.read_csv('sp_500_stocks.csv')

In [7]:
from loging_secrets import getToken

In [8]:
base_url = 'https://api.iex.cloud/v1'
symbol = 'aapl'
IEX_CLOUD_API_TOKEN = getToken()
quote_url = f'/data/core/quote/{symbol}'
token_var = f'/?token={IEX_CLOUD_API_TOKEN}'
api_url = base_url + quote_url + token_var

In [9]:
data = requests.get(api_url)

In [10]:
stock_data = data.json()[0]

In [11]:
my_columns = [ 'Ticker', 'Stock Price', 'Market Capitalization', 'Number of Shares to buy' ]

In [12]:
price = stock_data['latestPrice']
market_cap = stock_data['marketCap']
dataframe = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns = my_columns)

In [13]:
final_frame = pd.DataFrame([], columns = my_columns)

for index,stock in stocks.iterrows():
    symbol = stock['Ticker']
    data = requests.get(api_url)
    stock_data = data.json()[0]
    
    price = stock_data['latestPrice']
    market_cap = stock_data['marketCap']
    
    frame = pd.DataFrame([[symbol, price, market_cap, 'N/A']], columns = my_columns)
    final_frame = pd.concat([final_frame, frame], ignore_index=True)

    if index == 10 : break


C:\Users\elenp\AppData\Local\Temp\ipykernel_9936\3861297761.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_frame = pd.concat([final_frame, frame], ignore_index=True)


In [14]:
def chunks(lst, n):
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [15]:
symbol_groups = list(chunks(stocks['Ticker'].values.tolist()[:100], 10))

symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

final_frame = pd.DataFrame([], columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_url = base_url + f'/data/core/QUOTE/{symbol_string}' + token_var
    data = requests.get(batch_api_url) #list of objects

    for obj in data.json():
        frame = pd.DataFrame([[obj['symbol'], obj['latestPrice'], obj['marketCap'], 'N/A']], columns = my_columns)
        final_frame = pd.concat([final_frame, frame], ignore_index=True)

C:\Users\elenp\AppData\Local\Temp\ipykernel_9936\3639811771.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_frame = pd.concat([final_frame, frame], ignore_index=True)


In [16]:
portfolio_size = 1000000

position_size = portfolio_size/len(final_frame.index)

for i in range(0, len(final_frame)):
    final_frame.iloc[i, 3] = math.floor(position_size/final_frame.iloc[i, 1])

writer = pd.ExcelWriter('recommended_trades.xlsx', engine = 'xlsxwriter')
final_frame.to_excel(writer, 'Recommended Trades', index = False)

C:\Users\elenp\AppData\Local\Temp\ipykernel_9936\1689447475.py:9: FutureWarning: Starting with pandas version 3.0 all arguments of to_excel except for the argument 'excel_writer' will be keyword-only.
  final_frame.to_excel(writer, 'Recommended Trades', index = False)


In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_format = writer.book.add_format({'font_color':font_color, 'bg_color':background_color, 'border':1})
dolar_format = writer.book.add_format({'num_format':'$0.00', 'font_color':font_color, 'bg_color':background_color, 'border':1})
integer_format = writer.book.add_format({'num_format':'0', 'font_color':font_color, 'bg_color':background_color, 'border':1})

In [18]:
column_formats = {
    'A': ['Ticker', string_format], 
    'B': ['Stock Price', dolar_format], 
    'C': ['Market Capitalization', dolar_format], 
    'D': ['Number of Shares to buy', integer_format]
}

for col in column_formats.keys():
    writer.sheets['Recommended Trades'].set_column(f'{col}:{col}', 18, column_formats[col][1])


In [19]:
writer.close()